In [39]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point, mapping

In [23]:
admin2 = gpd.read_file('gadm36_ETH_shp/gadm36_ETH_3.shp')
admin2['Gadm Name 2'] = admin2['NAME_2']
admin2['Gadm Name 1'] = admin2['NAME_1']

In [24]:
pts3 = admin2.centroid.unary_union

from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 

def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 2'].get_values()[0]


def near_gadm1(geom, pts=pts3):
    if geom.type == Polygon:
        point = geom.centroid
    else:
        point = geom
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 1'].get_values()[0]

In [4]:
df = pd.read_csv('apsim.txt', sep='\t')

In [7]:
df['Latitude'] = df.CellId.apply(lambda x: float(x.split("_")[0])/1000)
df['Longitude'] = df.CellId.apply(lambda x: float(x.split("_")[1])/1000)

In [17]:
def parse_crop_season(crop_x_season):
    if 'Belg' in crop_x_season:
        season = 'Belg'
        crop = crop_x_season.split(season)[1]
    elif 'Meher' in crop_x_season:
        season = 'Meher'
        crop = crop_x_season.split(season)[1]
    elif 'Main' in crop_x_season:
        season = 'Main'
        crop = crop_x_season.split(season)[1]        
    return season, crop

In [18]:
df['Crop'] = df.Crop_x_Season.apply(lambda x: parse_crop_season(x)[1])
df['Season'] = df.Crop_x_Season.apply(lambda x: parse_crop_season(x)[0])

In [33]:
df['Geometry'] = df.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

In [40]:
df['Gadm Name 1'] = df.Geometry.apply(lambda x: near_gadm1(x))
df['Gadm Name 2'] = df.Geometry.apply(lambda x: near(x))

In [48]:
yield_cols = [i for i in df.columns if 'yield' in i]
other_cols = [i for i in df.columns if 'yield' not in i]

In [52]:
other_cols + [yield_cols[0]]

['CellId',
 'Household_Type',
 'Crop_x_Season',
 'Latitude',
 'Longitude',
 'Crop',
 'Season',
 'Geometry',
 'Gadm Name 1',
 'Gadm Name 2',
 'yield_r01']

In [59]:
df_combined = pd.DataFrame()
for y in yield_cols:
    d_ = df[other_cols + [y]]
    d_['Yield'] = d_[y]
    del(d_[y])
    d_['Climate Realization'] = y.split('_')[1]
    df_combined = df_combined.append(d_)

/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
df_combined

,CellId,Household_Type,Crop_x_Season,Latitude,Longitude,Crop,Season,Geometry,Gadm Name 1,Gadm Name 2,Yield,Climate Realization
0,10125_35625,FT_CRP,BelgBarley,10.125,35.625,Barley,Belg,POINT (35.625 10.125),Benshangul-Gumaz,Kemashi,0.00,r01
1,10125_35625,FT_MXD,BelgBarley,10.125,35.625,Barley,Belg,POINT (35.625 10.125),Benshangul-Gumaz,Kemashi,0.00,r01
2,10125_35875,FT_CRP,BelgBarley,10.125,35.875,Barley,Belg,POINT (35.875 10.125),Benshangul-Gumaz,Kemashi,0.06,r01
3,10125_35875,FT_MXD,BelgBarley,10.125,35.875,Barley,Belg,POINT (35.875 10.125),Benshangul-Gumaz,Kemashi,0.06,r01
4,10125_36375,FT_CRP,BelgBarley,10.125,36.375,Barley,Belg,POINT (36.375 10.125),Oromia,Misraq Wellega,0.48,r01
5,10125_36375,FT_MXD,BelgBarley,10.125,36.375,Barley,Belg,POINT (36.375 10.125),Oromia,Misraq Wellega,0.48,r01
6,10125_36625,FT_CRP,BelgBarley,10.125,36.625,Barley,Belg,POINT (36.625 10.125),Oromia,Misraq Wellega,2.03,r01
7,10125_36625,FT_MXD,BelgBarley,10.125,36.625,Barley,Belg,POINT (36.625 10.125),Oromia,Misraq Wellega,2.03,r01
8,10125_36875,FT_CRP,BelgBarley,10.125,36.875,Barley,Belg,POINT (36.875 10.125),Oromia,Misraq Wellega,2.34,r01
9,10125_36875,FT_MXD,BelgBarley,10.125,36.875,Barley,Belg,POINT (36.875 10.125),Oromia,Misraq Wellega,2.34,r01


In [61]:
df_combined.groupby(['Climate Realization']).CellId.count()

Climate Realization
r01    14053
r02    14053
r03    14053
r04    14053
r05    14053
r06    14053
r07    14053
r08    14053
r09    14053
r10    14053
r11    14053
r12    14053
r13    14053
r14    14053
r15    14053
r16    14053
r17    14053
r18    14053
r19    14053
r20    14053
r21    14053
r22    14053
r23    14053
r24    14053
r25    14053
r26    14053
r27    14053
r28    14053
r29    14053
r30    14053
Name: CellId, dtype: int64

In [62]:
df_combined.to_csv('apsim.csv', index=False)

In [65]:
df_combined.Yield.max()

9.46

In [66]:
df_combined.Yield.min()

0.0